# Autocorrect (Viterbi)
#### Authors:
v1.0 (2017 Spring) Tavor Baharav, Kabir Chandrasekhar, Sinho Chewi, Andrew Liu, Kamil Nar, David Wang, and Kannan Ramchandran

v1.1 (2017 Fall) Sinho Chewi, Avishek Ghosh, Chen Meng, Abhay Parekh, and Jean Walrand

For this option, you will be implementing autocorrect using the Viterbi algorithm.

## Hidden Markov Model

We will model the English language as a HMM. The state space is the set of all English words. The *hidden states* are the intended words of the writer, while the *emissions* are the actual words written down, which include typos. As an example, suppose an author intends to write the sentence "the dog barks". The sequence of hidden states is

$$X(0) = \text{the},$$
$$X(1) = \text{dog},$$
$$X(2) = \text{barks}.$$

However, the author is prone to making errors while typing, so the emissions (sometimes called observations) are

$$Y(0) = \text{the},$$
$$Y(1) = \text{dog},$$
$$Y(2) = \text{barjs}.$$

The Viterbi algorithm gives the most likely sequence of hidden states, i.e. the most likely sequence of intended words which produced the typo-filled output text.

In the field of natural language processing (NLP), the literature often mentions *$n$-grams* models. A *unigram* is a single word. A *bigram* is a pair of words which appear consecutively in a text.

You will use the unigram counts to determine the relative frequencies of words. The word frequencies specify the initial distribution of the HMM.

The bigram counts are used to infer word-to-word transition probabilities.

## Emission Model: Edit Distance

The emission probabilities are based on a measure of string dissimilarity known as *edit distance* or *Levenshtein distance*. The edit distance between two strings $s_1$ and $s_2$ (with lengths $m$ and $n$ respectively) is defined by the recursive equations

$$d(s_1, s_2) =
\begin{cases}
  m & \text{if } s_2 \text{ is empty}, \\
  n & \text{if } s_1 \text{ is empty}, \\
  \min
  \begin{cases}
    d(s_1(2, \dotsc, m), s_2) + 1 \\
    d(s_1, s_2(2, \dotsc, n)) + 1 \\
    d(s_1(2, \dotsc, m), s_2(2, \dotsc, n)) + \mathbf{1}\{s_1(1) \ne s_2(1)\}
  \end{cases}
  & \text{otherwise}.
\end{cases}
$$
The notation is as follows: $s_1(1)$ represents the first character in the string; and $s_1(2, \dotsc, n)$ represents the substring which starts at the second character and ends at the $n$th character of $s_1$. Similarly for $s_2$.

Here is some intuition for edit distance: it represents the minimum number of operations to change $s_1$ into $s_2$, where an operation is either
- adding a new character,
- deleting an existing character,
- or changing one character into another.

You can simply think of edit distance as a way to measure the distance between two strings. An example is shown below, demonstrating that the edit distance between "elephant" and "relevant" is 3, which is achieved by taking "relevant", deleting the r, replacing the v with a p, and adding an h immediately after.

<img src = "https://hsto.org/storage2/74b/c0f/a85/74bc0fa858652701ff47bfd125c83eeb.png">

The emission model is given by a Poisson distribution. If the hidden state $x$ and the emission $y$ have edit distance $d(x, y) = k$, then the emission probability is

$$Q(x, y) = \frac{e^{-\lambda} \lambda^k}{k!}.$$

The larger the edit distance, the smaller the emission probability, which aligns with our intuition.

## Dynamic Programming

Ensure that you implement the edit distance function using dynamic programming. Dynamic programming means solving the edit distance equations iteratively: start by computing the edit distance between $s_1(m)$ and $s_2(n)$; and then the edit distance between $s_1(m-1, m)$ and $s_2(n)$; and then the edit distance between $s_1(m)$ and $s_2(n-1, n)$; and so on. The key idea is to solve the equations backwards in order to avoid computationally prohibitive recursive calls that branch out into more recursive calls. As a benchmark, our implementation allows us to correct a sentence of three words in roughly 15 seconds.

A concrete example of a dynamic programming problem is to find the longest path in a DAG (directed acyclic graph). Suppose you are given a DAG $G = (V, E)$, and we wish to find the longest path in the DAG. Then, let the vertices be $v_1,\dotsc,v_n$, where $n := |V|$ and the order in which the vertices are listed is compatible with the DAG structure in the sense that if $1 \le i < j \le n$ are positive integers, then the edge $(v_j, v_i)$ does not appear in the graph. (Sorting the vertices in this way is possible because we are working with a DAG; this is known as a topological sort.) Starting from vertex $v_i$, where $i \in \{1,\dotsc,n\}$, let $L(v_i)$ denote the longest path in the DAG using the nodes $\{v_i,v_{i+1},\dotsc,v_{n-1},v_n\}$, starting from $v_i$. Then, we can write the recurrence $L(v_i) = 1 + \max\{L(v_j) : (v_i, v_j) \in E\}$, where $L(v_n) := 1$. Now we can solve these equations backwards: solve for $L(v_n)$ first, and then once $L(v_n)$ is known, we can solve for $L(v_{n-1})$. Next, solve for $L(v_{n-2})$ now that $L(v_{n-1})$ and $L(v_n)$ are known, and continue. The point is that we store the computed values of $L$ so we do not have to recompute them in each iteration, which leads to huge reductions in the runtime of the algorithm.

## Data Source

We will be using data from the source http://norvig.com/ngrams/ for the unigram and bigram counts.

We took the unigrams from the file "count_1w100k.txt" and the bigrams from the file "count_2w.txt".

## Implementation Advice

You do not have to follow these guidelines, but they may make your life a little easier. For starters, here is a short checklist of what to implement:
- load in the data, compute the unigram and bigram frequencies, and create matrices containing log probabilities (we recommend using log probabilities rather than actual probability values to avoid numerical underflow)
- a function which takes two strings as inputs and returns the edit distance between them
- a function which takes two strings as inputs and returns an emission probability
- implement the Viterbi algorithm

Make sure to test your implementation starting with a small list of words, because running the algorithm on a large set of words can be very time-consuming.

You will probably not want to use all of the words in the dataset, because the performance will not be too fast. Instead, consider limiting the number of words you use to a small subset of the dataset, based on the most frequently appearing words. (We recommend limiting the number of words to 10,000, but you can play around with this number too.)

If your implementation is not optimized, the performance may be very slow. Try to use NumPy's functions whenever possible because they are *vectorized* (optimized for high performance).

For the emission model, you may want to play around with different values of $\lambda$ to see which one works best. We tried using $\lambda = 0.01$.

Good luck and have fun!

In [1]:
import math
import numpy as np
import time

In [2]:
NUM_WORDS = 10000


pi_0 = np.zeros(NUM_WORDS)
word_to_id = {}
word_list = []


with open("count_1w100k.txt") as f:
    for i in range(NUM_WORDS):
        line = f.readline()
        tokens = line.lower().split()
        word = tokens[0]
        count = float(tokens[1])

        word_to_id[word] = i
        pi_0[i] = count
        word_list.append(word)


pi_0 = pi_0 / np.sum(pi_0)
log_pi_0 = np.log(pi_0)

In [3]:
P = np.zeros((NUM_WORDS, NUM_WORDS))


with open("count_2w.txt") as f:
    for line in f:
        tokens = line.lower().split()
        word1 = tokens[0]
        word2 = tokens[1]
        count = float(tokens[2])
        if word1 in word_to_id and word2 in word_to_id:
            P[word_to_id[word1], word_to_id[word2]] = count


P = P / np.sum(P)
log_P = np.log(P)

/usr/local/lib/python3.5/site-packages/ipykernel/__main__.py:15: RuntimeWarning: divide by zero encountered in log


In [4]:
def edit_distance(word1, word2):
    m = len(word1)
    n = len(word2)
    d = np.zeros((m + 1, n + 1))
    for i in range(m + 1):
        d[i, 0] = i
    for j in range(1, n + 1):
        d[0, j] = j
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            if word1[i - 1] == word2[j - 1]:
                d[i, j] = min(
                    d[i - 1, j] + 1, d[i, j - 1] + 1, d[i - 1, j - 1])
            else:
                d[i, j] = min(
                    d[i - 1, j] + 1, d[i, j - 1] + 1, d[i - 1, j - 1] + 1)
    return d[m, n]

In [5]:
RATE = 0.01


def viterbi(sentence):
    words = sentence.lower().split()
    count = len(words)

    log_Q = np.zeros((count, NUM_WORDS))
    for i in range(count):
        word = words[i]
        for j in range(NUM_WORDS):
            dist = edit_distance(word, word_list[j])
            log_Q[i, j] = dist * np.log(RATE) - np.log(math.factorial(dist))

    V = np.zeros((count, NUM_WORDS))
    pointers = np.zeros((count - 1, NUM_WORDS))

    for i in range(count - 2, -1, -1):
        for j in range(NUM_WORDS):
            dists = V[i + 1] - log_P[j] - log_Q[i + 1]
            index = np.argmin(dists)
            V[i, j] = dists[index]
            pointers[i, j] = index

    start_state = np.argmin(V[0] - log_pi_0 - log_Q[0])
    sentence = [word_list[int(start_state)]]
    curr_state = start_state
    for i in range(count - 1):
        curr_state = pointers[i, int(curr_state)]
        sentence.append(word_list[int(curr_state)])

    return " ".join(sentence)

In [6]:
start = time.time()
print(viterbi("whyy is th ski bluui"))
end = time.time()
print("Execution Time:", end - start)

why is the sky blue
Execution Time: 4.5631349086761475


Reference: https://www.cs.sfu.ca/~mori/courses/cmpt310/a5.pdf